In [1]:
import gensim
import os
import collections
import smart_open
import random
import multiprocessing
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint

In [2]:
#global variabls

directory_path = "/home/hao/AnacondaProjects/MLOntology/NCIt/"
data_path = directory_path + "data/"
vector_model_path = directory_path +"vectorModel/"
cnn_model_path = directory_path +"cnnModel/"

In [3]:
import re
def get_trailing_number(s):
    m = re.search(r'\d+$', s)
    return m.group() if m else None

In [4]:
#read class label file
#create mapping from id to labels  
#iso-8859-1 , encoding="iso-8859-1"
conceptLabelDict={}
errors=[]

def read_label(fname):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==3:
                conceptID = get_trailing_number(splitted[1])
                conceptLabelDict[conceptID] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)

label_file = data_path + "ontClassLabels_owl_ncit.txt"
read_label(label_file)
print(conceptLabelDict["4863"])
print(conceptLabelDict["115117"])
print(errors)


prostate carcinoma
stage ia esophageal cancer ajcc v7
[]


In [5]:
print(len(conceptLabelDict))

9955


In [6]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t", 2)
            
            line = splitted[2]
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                tagID = get_trailing_number(splitted[0])
                conceptMappingDict[i]= int(tagID)
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [tagID])

In [7]:
conceptMappingDict={}

train_file = data_path + "ontClassTopology_owl_ncit.txt"

train_corpus = list(read_corpus(train_file))

In [8]:
print(len(train_corpus))
train_corpus[1296:1299]

9955


[TaggedDocument(words=['neoplasm', 'by', 'special', 'category', 'neoplasm', 'by', 'obsolete', 'classification', 'hematopoietic', 'and', 'lymphoid', 'neoplasm', 'antiquated', 'lymphoma', 'by', 'working', 'formulation', 'antiquated', 'intermediate', 'grade', 'lymphoma', 'adult', 'non', 'hodgkin', 'lymphoma', 'grade', 'intermediate', 'grade', 'adult', 'non', 'hodgkin', 'lymphoma', 'stage', 'iii', 'intermediate', 'grade', 'adult', 'non', 'hodgkin', 'lymphoma', 'stage', 'iii', 'intermediate', 'grade', 'adult', 'non', 'hodgkin', 'lymphoma', 'stage', 'iii', 'intermediate', 'grade', 'adult', 'non', 'hodgkin', 'lymphoma'], tags=['8131']),
 TaggedDocument(words=['neoplasm', 'by', 'morphology', 'neoplasm', 'by', 'site', 'epithelial', 'neoplasm', 'head', 'and', 'neck', 'neoplasm', 'carcinoma', 'neck', 'neoplasm', 'malignant', 'head', 'and', 'neck', 'neoplasm', 'mucin', 'producing', 'carcinoma', 'laryngeal', 'neoplasm', 'malignant', 'neck', 'neoplasm', 'head', 'and', 'neck', 'carcinoma', 'mucoepide

In [9]:
cores = multiprocessing.cpu_count()

print(cores)
models = [
    # PV-DBOW 
    Doc2Vec(dm=0, dbow_words=1, vector_size=128, window=8, min_count=1, epochs=10, workers=cores),
    # PV-DM w/average
    Doc2Vec(dm=1, dm_mean=1, vector_size=128, window=8, min_count=1, epochs =10, workers=cores),
]


8


In [10]:
models[0].build_vocab(train_corpus)
print(str(models[0]))
models[1].reset_from(models[0])
print(str(models[1]))

Doc2Vec(dbow+w,d128,n5,w8,s0.001,t8)
Doc2Vec(dm/m,d128,n5,w8,s0.001,t8)


In [11]:
for model in models:
    %%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 52.9 s, sys: 1.21 s, total: 54.1 s
Wall time: 9.74 s
CPU times: user 23.3 s, sys: 3.6 s, total: 26.9 s
Wall time: 13.5 s


In [12]:
for i, model in enumerate(models):
    temp_path = vector_model_path + "model" + str(i)  # creates a temp file
    model.save(temp_path)

In [ ]:
model = Doc2Vec.load("model1") 

pprint(model.docvecs.most_similar([inferred_vector], topn=20))

In [ ]:
for model in models:
    print(str(model))
    inferred_vector = model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])
    pprint(model.docvecs.most_similar([inferred_vector], topn=20))

In [ ]:
for model in models:
    print(str(model))
    inferred_vector = model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])
    sims= model.docvecs.most_similar([inferred_vector], topn=20)
    for sim in sims:
        index = str(conceptMappingDict[sim[0]])
        print(conceptLabelDict[index], "score: ", sim[1])
    

In [ ]:
model.docvecs['133521']

In [ ]:
#gensim.utils.simple_preprocess??
gensim.models.doc2vec.TaggedDocument??

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=1, iter=5, alpha=0.025, min_alpha=0.025, sample = 1e-3)

gensim.models.doc2vec.Doc2Vec??

model.build_vocab(train_corpus)

len(model.wv.vocab)

%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)

# inferred_vector = model.infer_vector(['clinical', 'finding', 'evaluation', 'prevent', 'sampling', 'foot'])
inferred_vector = model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])


sims = model.docvecs.most_similar([inferred_vector], topn=10)
pprint(sims)

for sim in sims:
    print(train_corpus[sim[0]].words, "score: ", sim[1])
    
for sim in sims:
    print(conceptMappingDict[sim[0]], "score: ", sim[1])
  

from tempfile import mkstemp

fs, temp_path = mkstemp("gensim_temp")  # creates a temp file

model.save(temp_path)  # save the model

new_model = gensim.models.Doc2Vec.load(temp_path)

In [ ]:
for model in models:
    print(str(model))
    vec = [model.docvecs["Lady Gaga"] - model["american"] + model["japanese"]]
    pprint([m for m in model.docvecs.most_similar(vec, topn=11) if m[0] != "Lady Gaga"])

In [ ]:
print("Total: ",len(train_corpus))


ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    print("Processing: ", doc_id)
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

In [ ]:
import random, string

def randomStr(length):
   letters = string.ascii_lowercase
   return ''.join(random.choice(letters) for i in range(length))


In [ ]:
collections.Counter(ranks) 

In [17]:
import numpy as np
# vstack
from numpy import vstack
from numpy import hstack
from numpy import dstack

a1 = model.docvecs['133521']
# print(a1)
print(a1.shape)
print(a1)
a2 = model.docvecs['7918']
# print(a2)
print(a2.shape)
print(a2)

c = np.array((a1, a2))
c = c.T
print(c.shape)
print(c)



(128,)
[-0.02489698 -0.05052399  0.00643161 -0.00821158 -0.07351264 -0.02326442
 -0.01810085  0.01692173 -0.01278219 -0.05133998  0.0141927   0.01293162
 -0.07734409  0.02647892 -0.03104847 -0.02024052  0.03489167  0.04426214
  0.08376317  0.04952111 -0.07212681 -0.01899134 -0.06598855 -0.04032938
 -0.01688349  0.03832373 -0.01088326 -0.01021543  0.04160389  0.02573159
  0.00571989  0.05141127 -0.0354414   0.03459821  0.03820553 -0.02879284
  0.00692179 -0.0354858  -0.0201057  -0.03791515  0.00135138  0.03036554
  0.00424715  0.05342467 -0.04247887  0.04498743 -0.02336542 -0.02821766
  0.02692588 -0.01006284  0.0009644   0.01714842  0.00076355  0.02668407
 -0.01811202 -0.04951641 -0.00836854 -0.04120313 -0.01667412 -0.03620613
  0.04859411  0.00914418 -0.05046156 -0.0243607   0.01927145  0.02327728
 -0.07443111  0.02873673 -0.00627365 -0.06441654 -0.00417977 -0.02179801
 -0.05945649  0.00454753 -0.06056052 -0.00290074  0.04977814  0.10757856
 -0.04604999 -0.04097356  0.02243086  0.0218

In [21]:
a2 = model.docvecs['90016']

print(a2)

[-4.80370857e-02 -9.78515819e-02 -8.97581037e-03  7.48429820e-03
  1.77877359e-02  5.54668531e-02 -5.60946241e-02 -1.96124590e-03
  9.07555372e-02 -3.62381302e-02 -3.52795143e-03 -7.80802369e-02
  1.79551076e-02  2.75645070e-02  1.44851336e-03 -2.55759321e-02
  2.94994228e-02 -5.23815341e-02  2.32474450e-02 -8.53188708e-03
  2.08726041e-02  4.03827522e-03 -1.38276204e-01 -1.61579885e-02
  7.18939826e-02  5.61598456e-03 -1.25992522e-02 -5.88448271e-02
 -5.52711450e-02  3.30105238e-02  5.60095273e-02  1.66105554e-01
  5.04797921e-02 -3.30153070e-02  3.03500965e-02 -3.69303524e-02
  3.13679166e-02 -5.91785982e-02  1.06321633e-01 -4.39745970e-02
 -3.88688445e-02  8.59545469e-02 -1.18709123e-02  3.91735919e-02
 -8.31311755e-03  1.17642939e-01 -5.03510237e-02  1.28419369e-01
  6.67862520e-02 -3.28338668e-02 -5.89939952e-03 -2.94710603e-02
  9.66856629e-03 -2.31381459e-03 -1.09145511e-02 -7.86113366e-02
  5.10773854e-04 -8.83352011e-02 -8.53827596e-02  4.02513556e-02
 -5.19542769e-03  1.18624